In [22]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from src.features import build_features

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)


In [24]:
COLS_DIAGS = ["Capítulo", "Sección", "Categoría", "Descripción"]

In [81]:
df_procesada = pd.read_csv("../data/interim/datos_limpios.csv", encoding="latin-1", sep=";")
df_procesada["FECHA 1º evaluación"] = pd.to_datetime(
    df_procesada["FECHA 1º evaluación"], format="%Y-%m-%d"
)
df_procesada["F NAC"] = pd.to_datetime(df_procesada["F NAC"], yearfirst=True)

edad_primera_consulta = (
    df_procesada["FECHA 1º evaluación"] - df_procesada["F NAC"]
) / np.timedelta64(1, "Y")

df_procesada["EDAD_PRIMERA_CONSULTA"] = edad_primera_consulta
df_procesada["RANGO_ETARIO_PRIMERA_CONSULTA"] = pd.cut(
    df_procesada["EDAD_PRIMERA_CONSULTA"], [0, 15, 20, 25, 40, np.inf]
).astype(str)

df_procesada = df_procesada.fillna("SO")


# Analisis para Congenitas

En este analisis se quieren responder obtener los siguientes insights:

- Frecuencia de diagnosticos por anio
- Frecuencia de edad de primera consulta por anio (grupo etario)
- Frecuencia de procedencia por anio
- Frecuencia de sexo por anio

In [88]:
desglose_sociodemografico = ["ANIO_PRIMERA_EVALUACION", "validacion_Region"]

variables_a_analizar = ["validacion", "RANGO_ETARIO_PRIMERA_CONSULTA", "SEXO"]

In [89]:
resultado_sociodemo = build_features.obtener_dfs_para_desglose_sociodemografico(
    df_procesada, desglose_sociodemografico, variables_a_analizar
)

In [90]:
print(df_procesada["validacion"].value_counts().sum())
print(resultado_sociodemo["RANGO_ETARIO_PRIMERA_CONSULTA"]["conteo"].sum())
print(resultado_sociodemo["SEXO"]["conteo"].sum())
print(resultado_sociodemo["validacion"]["conteo"].sum())

3561
3561
3561
3561


- De los diccionarios obtenidos se puede ver que existen diferencias en las cantidades totales
de casos. Esto se debe a valores faltantes en cada una de las variables agregadas (ANIO_PRIMERA_EVALUACION,
Region, Rango Etario y Sexo)

- Esta diferencia solamente existe cuando se eliminan los valores faltantes. Si es que se convierten
a str tales valores, o se rellenan los Na (ej: con "SO"), entonces se obtiene la misma cantidad de registros.

In [93]:
with pd.ExcelWriter("../data/processed/tableros_powerbi/distribucion_sociodemografica.xlsx") as file:
    for nombre_hoja, df_a_guardar in resultado_sociodemo.items():
        df_a_guardar.to_excel(file, sheet_name=nombre_hoja)